<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/05_Transformers_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generación de Texto usando Transformers

Este notebook demuestra cómo usar modelos Transformer para generar texto en español.

Utilizaremos el modelo GPT2 específicamente ajustado para español.

##1) Importar librerías necesarias

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

##2) Cargar el modelo y tokenizador
Usamos 'datificate/gpt2-small-spanish', un modelo más ligero entrenado para español

In [2]:
model_name = "datificate/gpt2-small-spanish"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

##3) Función para generar texto

| Parámetro                                 | Descripción                                                                                                                                                             |
| ----------------------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **`inputs["input_ids"]`**                 | Entrada codificada con el tokenizador. Representa el texto base sobre el que se generará la continuación.                                                               |
| **`max_length`**                          | Longitud máxima (en tokens) de la secuencia generada, incluyendo los tokens de entrada. Limita el tamaño total del texto generado.                                      |
| **`num_return_sequences`**                | Número de secuencias distintas que el modelo debe generar para cada entrada. Por ejemplo, `3` generará tres variantes de texto.                                         |
| **`no_repeat_ngram_size=2`**              | Impide que se repitan bigramas (secuencias de 2 palabras seguidas) en el texto generado. Reduce repeticiones molestas.                                                  |
| **`do_sample=True`**                      | Habilita la generación con muestreo aleatorio en lugar de usar solo la probabilidad más alta (*greedy decoding*). Necesario para usar `top_k`, `top_p` y `temperature`. |
| **`top_k=50`**                            | En cada paso, el modelo considera solo los **50 tokens más probables** (con mayor probabilidad). Reduce el espacio de decisión y evita resultados muy dispersos.        |
| **`top_p=0.95`**                          | Usado en *nucleus sampling*: considera los tokens más probables cuya suma acumulada de probabilidad sea ≤ 0.95. Ofrece más variedad que `top_k` solo.                   |
| **`temperature=0.7`**                     | Controla la aleatoriedad: valores < 1 hacen la salida más conservadora y enfocada; valores > 1 la hacen más creativa e impredecible.                                    |
| **`pad_token_id=tokenizer.eos_token_id`** | Define el token de padding para completar la secuencia si no llega al `max_length`. Aquí se usa el token de fin de secuencia como relleno.                              |



In [3]:
def generar_texto(prompt, max_length=100, num_return_sequences=1):
    # Tokenizar el texto de entrada
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generar texto
    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decodificar y mostrar los resultados
    for i, output in enumerate(outputs):
        texto_generado = tokenizer.decode(output, skip_special_tokens=True)
        print(f"\nTexto generado {i+1}:")
        print(texto_generado)
        print("-" * 50)

##4) Probar diferentes tipos de generación

In [4]:
# Ejemplo 1: Completar una historia
print("=== Completando una historia ===")
prompt_historia = "Había una vez un científico que descubrió"
generar_texto(prompt_historia, max_length=150)

# Ejemplo 2: Generar una receta
print("\n=== Generando una receta ===")
prompt_receta = "Para preparar una tortilla española necesitas"
generar_texto(prompt_receta, max_length=200)

# Ejemplo 3: Múltiples variaciones
print("\n=== Generando múltiples variaciones ===")
prompt_variaciones = "El futuro de la inteligencia artificial"
generar_texto(prompt_variaciones, max_length=100, num_return_sequences=3)

=== Completando una historia ===

Texto generado 1:
Había una vez un científico que descubrió su talento y se le dio una beca para estudiar química. El presidente de la Universidad de Harvard, John B. W. Kline, le ofreció una cátedra en química en la Escuela de Medicina Harvard. La segunda mujer fue nombrada directora en 1938. 

En 1943, fue contratada como asistente del laboratorio de Klicker y, en 1945, ayudó a fundar el Laboratorio Nacional de Química (NSQ) en Washington D. C., en donde se convirtió en una de las primeras mujeres en ser nombradas directora. En 1948, se mudó a los Estados Unidos para trabajar en el Departamento de Ingeniería de Sistemas de Información para el Instituto de Tecnología de Massachusetts. Allí, trabajó en un estudio de campo de energía eléctrica y
--------------------------------------------------

=== Generando una receta ===

Texto generado 1:
Para preparar una tortilla española necesitas un gran esfuerzo. En primer lugar, si el animal que se desea se e

##5) Funcion Interactiva

In [5]:
def generar_texto_interactivo():
    prompt = input("\nEscribe un comienzo para generar texto: ")
    longitud = int(input("¿Cuántos caracteres quieres generar? (recomendado: 100-200): "))
    variaciones = int(input("¿Cuántas variaciones quieres ver? (1-3): "))

    print("\nGenerando texto...")
    generar_texto(prompt, max_length=longitud, num_return_sequences=variaciones)

# Probar la función interactiva
print("\n=== Prueba la generación de texto ===")
generar_texto_interactivo()


=== Prueba la generación de texto ===

Escribe un comienzo para generar texto: Ests profesor me
¿Cuántos caracteres quieres generar? (recomendado: 100-200): 100
¿Cuántas variaciones quieres ver? (1-3): 2

Generando texto...

Texto generado 1:
Ests profesor me dijo que se sentía como si fuera un niño y que estaba a punto de morir. Pero luego de hacer algunas de esas cosas, fue a la casa de su familia. En el camino, estaba en el coche, y luego fue atropellado por un automóvil. Se dio cuenta de que era el hijo de la familia, pero no puedo decir lo que pasó con él, por lo menos me ha dicho que había estado en los años anteriores. Era muy divertido.

En julio
--------------------------------------------------

Texto generado 2:
Ests profesor me cuesta mucho, pero me gusta el juego de béisbol y el béisbol. Es como si me fuera un gran fan de la serie de televisión de TNT.

Eskimowitz se graduó en la Universidad de Chicago, donde se especializó en economía, como la cual se ha especializado en